

![](https://www.python.org/static/img/python-logo.png)



# Python Dictionaries
---

In python, dictionaries are hash maps - data structures that use a hashing function to map keys to locations in memory where values are stored.

## Hashing Function
---

A hashing function is used to deterministically return a consistent output for a given input. The output of this function determines the place in memory where the value is stored. A consequence of this, is that hash keys must be immutable, as the output from the hash function could unexpectedly change if, after storing a key value pair in the structure, the key were to change.

We can see why this is a problem in python's hashing function for a tuple, if it were to be used on a list:

In [3]:
import ctypes

class hashable_list(list):
    c_mul = lambda self, a, b: ctypes.c_int32((a * b) & 0xffffffff).value
    def __hash__(self):
        value = 0x345678
        for item in self:
            value = self.c_mul(1000003, value) ^ hash(item)
        value = value ^ len(self)
        if value == -1:
            value = -2
        return value

our_dict = {}
key = hashable_list([0, 0])
our_dict[key] = True
print('our value stored at our_dict[key]:', our_dict[key])
hashed_key = key.__hash__()
key[1] = 1
same_key_rehashed = key.__hash__()
print('Is our key the same as it was?', hashed_key == same_key_rehashed, hashed_key, same_key_rehashed)
print(our_dict[key])

our value stored at our_dict[key]: True
Is our key the same as it was? False 218750522 218750523


KeyError: [0, 1]

### Unhashable Types
---

By default, python provides support for hashing immutable structures. Other data structures are unhashable types.

In [45]:
cache = {}
cache[0] = 0
cache[(0, 0)] = 0
cache['a'] = 0
cache[frozenset(['alpha', 'bravo'])] = 0
cache[[0, 0]] = 0

TypeError: unhashable type: 'list'

In [46]:
cache = {set([1, 2]): 0}

TypeError: unhashable type: 'set'

In [47]:
cache = {{'a': True}: True}

TypeError: unhashable type: 'dict'

## Collision-Handling Schemes
---

While an input will always hash to the same output, there's no guarantee that two unique inputs won't hash to the same output.

With a small dataset and the strength of the hashing functions in use, collisions are unlikely. We could see the potential for a collision in a simple scenario using a really bad hashing function, however:

In [2]:
bad_hash = lambda letters: int(''.join([str(ord(c) - 96) for c in letters]))
first_hash = bad_hash('aa')
second_hash = bad_hash('k')
print('Do we have a collision?', first_hash == second_hash, first_hash, second_hash)

Do we have a collision? True 11 11


### Separate Chaining
---

![](https://he-s3.s3.amazonaws.com/media/uploads/0e2c706.png)

One method of addressing the issue of collisions is to maintain at every key a linked list of values stored at that key. While simpler to implement than other collision-handling methods, this approach results in O(n) operations (get, set, delete) in the worst case, n referring to the size of the bucket, or the number of collisions for that hashed key. This is because with each operation referencing this bucket of collisions, we must iterate over each item, looking for our target. It's also more costly in terms of space than alternatives that don't utilize extra data structures, in this case, our repeated use of additional linked lists.

### Open Addressing
---

![](https://cdn-images-1.medium.com/max/1600/1*xN0omiiMDelgCQmmg7zv9Q.png)

#### Linear Probing
---
An alternative approach is to use no extra data structure, but store new values in, in the event of a hashed key collision, the next available space. This approach saves space, but group items into contiguous runs, which can cause overlap between two hashed keys that otherwise would have no collision. This is more common when more than half of the cells are occupied.

#### Quadratic Probing
---
Linear probing, but instead of placing in the next available space, place it in the next available space at increments that increase exponentially with each attempt. This alleviates grouping of linear probing, but creates a secondary clustering and the table is still filled in a non-uniform manner.

#### Double Hashing
---
Another approach is to use a secondary hashing function to redistribute the keys of a collision.

#### Python's Approach
---
Python's approach, as of the publication of the 2013 edition of Goodrich's _Data Structure and Algorithms in Python_, is to increment the hashed key by a function of i (the number of attempts at finding an empty cell) % n (the size of the table). `f(i)` uses a pseudo-random number generator that provides a somewhat arbitrary but repeatable sequence of cell attempts.

### Tradeoffs
---
Geeksforgeeks.org provides a summary of the tradeoffs between Separate Chaining and Open Addressing, and some thoughts on the more common open-addressing approaches. Jupyter notebooks 

| Seperate Chaining                                                                                       | Open Addressing                                                                              |
|---------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------|
| Chaining is simpler to implement.                                                                       | Open Addressing requires more computation.                                                   |
| In chaining, Hash table never fills up, we can always add more elements to chain.                       | In open addressing, table may become full.                                                   |
| Chaining is Less sensitive to the hash function or load factors.                                        | Open addressing requires extra care for to avoid clustering and load factor.                 |
| Chaining is mostly used when it is unknown how many and how frequently keys may be inserted or deleted. | Open addressing is used when the frequency and number of keys is known.                      |
| Cache performance of chaining is not good as keys are stored using linked list.                         | Open addressing provides better cache performance as everything is stored in the same table. |
| Wastage of Space (Some Parts of hash table in chaining are never used).                                 | In Open addressing, a slot can be used even if an input doesn’t map to it.                   |
| Chaining uses extra space for links.                                                                    | No links in Open addressing                                                                  |

